In [1]:
pip install yt-dlp


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install yt-dlp



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install openai-whisper gradio google-api-python-client "imageio[ffmpeg]"


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import gradio as gr
import os
import yt_dlp
import whisper
from googleapiclient.discovery import build

# Helper Function: Fetch YouTube Videos
def fetch_youtube_videos(subject):
    api_key = os.getenv("YOUTUBE_API_KEY")  # Set your YouTube API Key
    youtube = build("youtube", "v3", developerKey=api_key)

    try:
        request = youtube.search().list(
            q=subject,
            part="snippet",
            maxResults=3,
            order="viewCount",
            type="video"
        )
        response = request.execute()

        videos = []
        for item in response.get("items", []):
            videos.append({
                "Title": item["snippet"]["title"],
                "Channel": item["snippet"]["channelTitle"],
                "Published At": item["snippet"]["publishedAt"],
                "Description": item["snippet"]["description"],
                "Video Link": f"https://www.youtube.com/watch?v={item['id']['videoId']}"
            })
        
        result = "\n\n".join(
            [f"**Title**: {video['Title']}\n**Channel**: {video['Channel']}\n**Link**: {video['Video Link']}"
             for video in videos]
        )
        return result

    except Exception as e:
        return f"Error: {str(e)}"

# Helper Function: Transcribe Audio
def download_audio(youtube_url, output_path="audio"):
    ydl_opts = {
        "format": "bestaudio/best",
        "postprocessors": [{
            "key": "FFmpegExtractAudio",
            "preferredcodec": "mp3",
            "preferredquality": "192",
        }],
        "outtmpl": output_path,
        "quiet": True,
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([youtube_url])
        return f"{output_path}.mp3"
    except Exception as e:
        return f"Error during download: {e}"

def transcribe_audio(file_path, model_type="base"):
    model = whisper.load_model(model_type)
    result = model.transcribe(file_path)
    return result["text"]

def transcribe_youtube_video(youtube_url):
    try:
        audio_file = download_audio(youtube_url)
        transcript = transcribe_audio(audio_file)
        if os.path.exists(audio_file):
            os.remove(audio_file)  # Clean up audio file
        return transcript
    except Exception as e:
        return f"Error: {str(e)}"

# Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("# YouTube Content Analyzer")
    
    with gr.Tab("Video Retriever"):
        gr.Markdown("## Search for YouTube Videos")
        subject_input = gr.Textbox(label="Subject", placeholder="Enter a topic (e.g., Machine Learning)")
        video_output = gr.Markdown()
        fetch_button = gr.Button("Fetch Videos")
        fetch_button.click(fetch_youtube_videos, inputs=[subject_input], outputs=[video_output])
    
    with gr.Tab("Video Transcription"):
        gr.Markdown("## Transcribe a YouTube Video")
        youtube_url_input = gr.Textbox(label="YouTube URL", placeholder="Paste the video link here")
        transcription_output = gr.Textbox(label="Transcription", lines=10)
        transcribe_button = gr.Button("Transcribe Audio")
        transcribe_button.click(transcribe_youtube_video, inputs=[youtube_url_input], outputs=[transcription_output])

if __name__ == "__main__":
    app.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Traceback (most recent call last):
  File "c:\Users\reddo\anaconda3\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\reddo\anaconda3\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\reddo\anaconda3\Lib\site-packages\gradio\blocks.py", line 2042, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\reddo\anaconda3\Lib\site-packages\gradio\blocks.py", line 1589, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\reddo\anaconda3\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
 